In [9]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults
import os

load_dotenv()

# client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
search = TavilySearchResults(max_results=1)
GROQ_MODEL = os.getenv("GROQ_MODEL")
GROQ_API = os.getenv("GROQ_API_KEY")
llm = ChatGroq(api_key=GROQ_API, model=GROQ_MODEL)


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


In [5]:
embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2")

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = Chroma.from_documents(
    documents=documents,
    embedding=embed_model,
    persist_directory="./chroma_db"
)
retriever = vector.as_retriever()


In [6]:
retriever.invoke("how to upload a dataset")[0]


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Document(metadata={'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith'}, page_content='ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright Â© 2025 LangChain, Inc.')

In [11]:
from langchain.tools.retriever import create_retriever_tool
from langchain_core.messages import HumanMessage


In [8]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)


In [10]:
tools = [search, retriever_tool]


In [13]:
response = llm.invoke([HumanMessage(content="hi!")])
response.content


"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [14]:
model_with_tools = llm.bind_tools(tools)


In [15]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Hi!'}, 'id': 'call_7rvz', 'type': 'tool_call'}]


In [17]:
response = model_with_tools.invoke(
    [HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_d56w', 'type': 'tool_call'}]


In [19]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


d:\Anaconda\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [21]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)


In [22]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)


In [23]:
agent_executor.invoke({"input": "hi!"})


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


{'input': 'hi!',
 'output': "LangSmith is a comprehensive solution for developers and data scientists working with NLP models. It offers a robust environment for creating, testing, and deploying language models in a scalable and maintainable manner. By leveraging LangChain's capabilities, Langsmith simplifies the process of chaining multiple language models and other NLP components to build powerful applications."}

In [24]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})


{'input': 'how can langsmith help with testing?',
 'output': 'It seems that LangSmith can be very helpful with testing, especially when it comes to testing LLM applications. The tool call id "call_3f12" yielded some interesting information about using Pytest and Vitest for LangSmith evaluations. It appears that LangSmith allows for logging feedback and comparing results over time to prevent regressions and ensure the best version of an application is deployed. The example provided even shows how to evaluate an application that generates SQL queries and logs the results to LangSmith.\n\nSo, to answer your original question, LangSmith can help with testing by providing a way to log feedback and compare results over time, allowing for more nuanced testing of LLM applications.'}

In [25]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Nice to meet you, Bob! How can I help you today?'}

In [26]:
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)


{'chat_history': [HumanMessage(content='hi! my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'input': "what's my name?",
 'output': 'Your name is Bob!'}

In [30]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [31]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)


In [32]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm Akash Sharma"},
    config={"configurable": {"session_id": "<foo>"}}
)


{'input': "hi! I'm Akash Sharma",
 'chat_history': [],
 'output': 'Nice to meet you, Akash Sharma!'}

In [33]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    config={"configurable": {"session_id": "<foo>"}},
)


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm Akash Sharma", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Akash Sharma!', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Akash Sharma!'}